<div style="color:#007acc; text-align:center; font-size:32px; font-weight:bold; font-family:Arial, sans-serif;">
  Explore Data Analysis and Transform Equipenhance Log Data
</div>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
df=pd.read_csv("../ingest_from_ITS/data/m952_equipenhance_2025-05-04_00-00-00_to_2025-06-19_00-00-00.csv",
               usecols=['@timestamp','user.id','event.its.properties.i_before_level','event.its.properties.i_after_level']
            ) 
df

C:\Users\GOSU.GOSU-TT-ZION1\AppData\Local\Temp\ipykernel_17456\79412123.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../ingest_from_ITS/data/m952_equipenhance_2025-05-04_00-00-00_to_2025-06-19_00-00-00.csv",


,@timestamp,event.its.properties.i_before_level,event.its.properties.i_after_level,user.id
0,2025-05-03T17:00:00.000Z,2,3,2407152127470964
1,2025-05-03T17:00:00.000Z,0,1,2501151225298848
2,2025-05-03T17:00:01.000Z,2,0,2503222323048092
3,2025-05-03T17:00:01.000Z,1,2,2412231548057710
4,2025-05-03T17:00:01.000Z,3,0,2407030933487570
...,...,...,...,...
6820886,2025-06-18T16:59:56.000Z,0,1,2409051732598555
6820887,2025-06-18T16:59:57.000Z,0,1,2503012137220422
6820888,2025-06-18T16:59:57.000Z,5,4,2502010237011465
6820889,2025-06-18T16:59:57.000Z,5,6,2407041513419618


In [3]:
df['user.id']=df['user.id'].astype(str)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820891 entries, 0 to 6820890
Data columns (total 4 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   @timestamp                           object
 1   event.its.properties.i_before_level  int64 
 2   event.its.properties.i_after_level   int64 
 3   user.id                              object
dtypes: int64(2), object(2)
memory usage: 208.2+ MB


In [5]:
df.describe()

,event.its.properties.i_before_level,event.its.properties.i_after_level
count,6820891.00,6820891.00
mean,1.69,2.00
std,1.72,1.68
min,0.00,-1.00
25%,0.00,1.00
50%,1.00,2.00
75%,3.00,3.00
max,14.00,16.00


In [6]:
df.isnull().sum()

@timestamp                             0
event.its.properties.i_before_level    0
event.its.properties.i_after_level     0
user.id                                0
dtype: int64

In [7]:
null_ratio = df.isnull().mean()
cols_to_drop = null_ratio[null_ratio > 0.95].index
print("column deleted:", list(cols_to_drop))
df = df.drop(columns=cols_to_drop)
df.shape

column deleted: []


(6820891, 4)

In [8]:
df['@timestamp'] = pd.to_datetime(df['@timestamp'], errors='coerce', utc=True)
df.shape

(6820891, 4)

In [9]:
len(df['user.id'].unique())

55138

#### **Visualize**

In [10]:
# user_id = 2206111830413361
# df_user = df[df['user.id'] == user_id].copy()

# df_user['level_change'] = df_user['event.its.properties.i_after_level'] - df_user['event.its.properties.i_before_level']

# plt.figure(figsize=(8,5))
# df_user['level_change'].value_counts().sort_index().plot(kind='bar', color='skyblue')
# plt.title(f'Num changes of user {user_id}')
# plt.xlabel('Level change (after - before)')
# plt.ylabel('Times')
# plt.grid(axis='y')
# plt.tight_layout()
# plt.show()

#### **Create features**

In [11]:
data=df
data = data.rename({'user.id': 'vopenid'}, axis=1)
now = pd.Timestamp('2025-06-26 00:00:00', tz='UTC') 

In [12]:
# time update
def last_update_time(group):
    last_time = group['@timestamp'].max()
    return {
        'last_update_time': last_time
    }

In [13]:
# number enhance
def total_attempts(group):
    return {
        'total_attempts': len(group)
    }

In [14]:
# total success enhance
def total_success_wt(group):
    success = (group['event.its.properties.i_after_level'] > group['event.its.properties.i_before_level']).sum()
    return {
        'total_success_wt': success
    }

In [15]:
# rate success enhance
def equipenhance_lucky(group):
    total = len(group)
    success = (group['event.its.properties.i_after_level'] > group['event.its.properties.i_before_level']).sum()
    return {
        'equipenhance_lucky': success / total if total > 0 else 0
    }

In [16]:
# streak fail longest
def max_consecutive_failures(group):
    fails = (group['event.its.properties.i_after_level'] <= group['event.its.properties.i_before_level']).tolist()
    max_fail = 0
    current = 0
    for f in fails:
        if f:
            current += 1
            if current > max_fail:
                max_fail = current
        else:
            current = 0
    return {
        'max_consecutive_failures': max_fail
    }


In [17]:
features_function=[
   last_update_time,
   total_attempts,
   total_success_wt,
   equipenhance_lucky,
   max_consecutive_failures,
]

In [18]:
def extract_group_features(group):
    result={}
    for func in features_function:
        try:
            result.update(func(group))
        except Exception as e:
            print(f"Error at feature: {func}. Error: {e}")
    return pd.Series(result)

df_res = data.groupby('vopenid').apply(extract_group_features).reset_index()
df_res

C:\Users\GOSU.GOSU-TT-ZION1\AppData\Local\Temp\ipykernel_17456\3747869921.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_res = data.groupby('vopenid').apply(extract_group_features).reset_index()


,vopenid,last_update_time,total_attempts,total_success_wt,equipenhance_lucky,max_consecutive_failures
0,2206111808105074,2025-06-15 14:53:06+00:00,2,2,1.00,0
1,2206111830413361,2025-06-16 06:01:59+00:00,79,51,0.65,5
2,2206112142566650,2025-06-17 03:34:01+00:00,14,11,0.79,1
3,2206131428405180,2025-06-13 03:56:37+00:00,1,1,1.00,0
4,2206141543270674,2025-06-10 14:35:30+00:00,2,0,0.00,2
...,...,...,...,...,...,...
55133,6220001341173,2025-06-08 06:12:13+00:00,4,3,0.75,1
55134,6220001406323,2025-06-17 14:14:40+00:00,5,2,0.40,2
55135,6220001449993,2025-06-13 07:30:53+00:00,4,3,0.75,1
55136,chungminhde,2025-06-18 11:14:52+00:00,16,11,0.69,2


In [20]:
df_res.to_csv("data/equipenhance_transform.csv",index=False, header=True)